In [1]:
import numpy as np
import joblib
from skimage.feature import hog
from skimage import color
from skimage.transform import pyramid_gaussian
from imutils.object_detection import non_max_suppression
from scipy.special import expit
import cv2

In [2]:
model = joblib.load('svm_model3.npy')

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

scale = 0
detections = []

(winW, winH) = (64,128)  # size of the sliding window
windowSize = (winW,winH)
downscale = 1.25 

In [3]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

In [6]:
img = cv2.imread('C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\detection\\inputs\\1\\184.png')
img = cv2.resize(img,(300,200))

In [7]:
for resized in pyramid_gaussian(img, downscale=downscale): # loop over each layer of the image that you take
    # loop over the sliding window for each layer of the pyramid
    for (x,y,window_) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
        # if the window does not meet our desired window size, ignore it!
        if window_.shape[0] != winH or window_.shape[1] !=winW or window_.shape[2]!=3: # ensure the sliding window has met the minimum size requirement
            continue
        window = color.rgb2gray(window_)
        #window = cv2.cvtColor(window_, cv2.COLOR_BGR2GRAY) #.rgb2gray(window_[:,:,:3])
        fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')  # extract HOG features from the window captured
        fds = fds.reshape(1, -1) # re shape the image to make a silouhette of hog
        pred = model.predict(fds) # use the SVM model to make a prediction on the HOG features extracted from the window
        
        if pred == 1:
            if expit(model.decision_function(fds))>=0.90: #model.decision_function(fds) > 1.5:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                print("Detection:: Location -> ({}, {})".format(x, y))
                print("Scale ->  {} | Confidence Score {} | Prob: {}\n".format(scale ,model.decision_function(fds),
                                                                              expit(model.decision_function(fds))))
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                   int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
                                      int(windowSize[1]*(downscale**scale))))
                
    scale+=1

Detection:: Location -> (30, 20)
Scale ->  23 | Confidence Score [3.29705529] | Prob: [0.96432765]

Detection:: Location -> (50, 20)
Scale ->  23 | Confidence Score [2.91674428] | Prob: [0.94866799]

Detection:: Location -> (70, 20)
Scale ->  23 | Confidence Score [2.41205048] | Prob: [0.91774161]

Detection:: Location -> (90, 20)
Scale ->  23 | Confidence Score [2.20941678] | Prob: [0.90109196]

Detection:: Location -> (60, 30)
Scale ->  23 | Confidence Score [2.42179848] | Prob: [0.91847451]

Detection:: Location -> (80, 30)
Scale ->  23 | Confidence Score [2.93968323] | Prob: [0.94977362]

Detection:: Location -> (140, 40)
Scale ->  23 | Confidence Score [2.41257774] | Prob: [0.9177814]

Detection:: Location -> (60, 50)
Scale ->  23 | Confidence Score [4.35214121] | Prob: [0.98728456]

Detection:: Location -> (210, 70)
Scale ->  23 | Confidence Score [2.40045741] | Prob: [0.91686218]

Detection:: Location -> (20, 0)
Scale ->  24 | Confidence Score [2.84347208] | Prob: [0.94498026]



In [8]:
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
sc = [score[0] for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
nms_rects = non_max_suppression(rects, probs = sc, overlapThresh=0.3)

detection confidence score:  [3.297055289310304, 2.0570444926431533, 2.9167442757767343, 2.412050483454074, 2.2094167810308356, 2.4217984762426914, 2.939683227285977, 2.0529073108730946, 2.4125777357920386, 1.8916673424122061, 1.83619632902814, 4.3521412070361825, 2.0047304212279453, 1.8539867918040924, 1.9491144567290553, 2.047090819672328, 2.4004574111101586, 2.843472080331523, 2.251584806672958, 2.2075432401555, 2.7292720870629488, 2.1029234888506227, 3.947671607426921, 2.9517514720197875, 4.20926427658979, 3.848583949269905, 2.0436891093743945, 2.0948363991207932, 3.8637954494324536, 2.113745214265828, 3.297055289310304, 2.9167442757767343, 2.412050483454074, 2.2094167810308356, 2.4217984762426914, 2.939683227285977, 2.4125777357920386, 4.3521412070361825, 2.4004574111101586, 2.843472080331523, 2.251584806672958, 2.2075432401555, 2.7292720870629488, 3.947671607426921, 2.9517514720197875, 4.20926427658979, 3.848583949269905, 3.8637954494324536]


In [22]:
rects

array([[ 10,   0,  74, 128],
       [ 20,   0,  84, 128],
       [ 10,  10,  74, 138],
       [ 10,  20,  74, 148],
       [ 20,  20,  84, 148],
       [ 40,  60, 104, 188],
       [125,  37, 205, 197]])

In [9]:
nms_sc = []
for i in range(rects.shape[0]):
    for j in range(nms_rects.shape[0]):
        if np.array_equal(rects[i], nms_rects[j]):
            nms_sc.append(sc[i])

In [10]:
nms_rects

array([[   60,    50,   124,   178],
       [10164,  8470, 21006, 30154],
       [21175,  2117, 34727, 29222],
       [  125,    12,   205,   172],
       [  210,    70,   274,   198],
       [35575, 11858, 46417, 33542]])

In [11]:
for i, (xA, yA, xB, yB) in enumerate(nms_rects):
    print(i)
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
    cv2.putText(img,'Human',(xA-2,yA-2),1,0.75,(255,255,0),1)

0
1
2
3
4
5


In [12]:
cv2.imshow('Detection', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
model = joblib.load('svm_model3.npy')

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

(winW, winH) = (64,128)  # size of the sliding window
windowSize = (winW,winH)
downscale = 1.25

In [15]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

In [16]:
def detect(img):
    scale = 0
    detections = []
    for resized in pyramid_gaussian(img, downscale=downscale): # loop over each layer of the image that you take
    # loop over the sliding window for each layer of the pyramid
        for (x,y,window_) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
            # if the window does not meet our desired window size, ignore it!
            if window_.shape[0] != winH or window_.shape[1] !=winW or window_.shape[2]!=3: # ensure the sliding window has met the minimum size requirement
                continue
            window = color.rgb2gray(window_)
            #window = cv2.cvtColor(window_, cv2.COLOR_BGR2GRAY) #.rgb2gray(window_[:,:,:3])
            fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')  # extract HOG features from the window captured
            fds = fds.reshape(1, -1) # re shape the image to make a silouhette of hog
            pred = model.predict(fds) # use the SVM model to make a prediction on the HOG features extracted from the window

            if pred == 1:
                if expit(model.decision_function(fds))>=0.90:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                    #print("Detection:: Location -> ({}, {})".format(x, y))
                    #print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                    detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), expit(model.decision_function(fds)),
                                       int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
                                          int(windowSize[1]*(downscale**scale))))
        scale+=1
    return detections

In [17]:
# importing libraries
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
#cap = cv2.VideoCapture('C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\detection\\videoplayback.mp4')

cap = cv2.VideoCapture(0)
# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

# Read until video is completed
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_res = cv2.resize(frame,(300,200))
        detections = detect(frame_res)
        rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
        sc = [score[0] for (x, y, score, w, h) in detections]
        sc = np.array(sc)
        nms_rects = non_max_suppression(rects, probs = sc, overlapThresh=0.3)
        nms_sc = []
        for i in range(rects.shape[0]):
            for j in range(nms_rects.shape[0]):
                if np.array_equal(rects[i], nms_rects[j]):
                    nms_sc.append(sc[i])
        nms_sc = np.array(nms_sc)
        
        for i, (xA, yA, xB, yB) in enumerate(nms_rects):
            cv2.rectangle(frame_res, (xA, yA), (xB, yB), (0,255,0), 2)
            cv2.putText(frame_res,'Human',(xA-2,yA-2),1,0.75,(255,255,0),1)
            print(nms_sc[i])
            
        cv2.imshow('frame', frame_res)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else:
        break
        
cap.release()
cv2.destroyAllWindows()


0.957789328776077
0.9704486316797872
0.9312274295354676
0.9591005031987316
0.9860022500585525
0.9806327861730264
0.9325644804403296
0.9522511174307254
0.9748141358862494
0.9236439375402665
0.9520616264625686
0.9036837241764731
0.9648980279395281
0.913444847194971
0.9605705261721993
0.9330804094996304
0.9124567953132977
0.932642274110466
0.979741780821027
0.939784206819394
0.9676190278751077
0.9173055999596768
0.9612146614686199
0.9277648362746053
0.9393343242991016
0.9449443046422201
0.9830175968917747
0.9335018689224741
0.9656208045943705
0.9526718612936019
0.9183340211246398
0.9516373880279752
0.9376628947068532
0.9474568591355822
0.9777591590581719
0.9070233593128145
0.9857714183074272
0.9111959629468953
0.9748431252215864
0.9686283643975245
0.9801399415243384
0.9253172776081089
0.9151633379941528
0.9583750457615737
0.9348193716496606
0.9632319073430883
0.9464098903414947
0.9258388743065451
0.9333173831319305
0.9067422421244282
0.9806802744430391
0.9858882870042992
0.922860657192135